# Setup

In [1]:
import transformers
import numpy as np
import torch
import get_pubmed_text as pmidUtils
import json

/w5home/lxu3/anaconda3/envs/kmGPT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x = ["23956253", "23444397", "28886926", "27701660", "19095139", "21095263"]

In [3]:
with open("../config.json", "r") as f:
	config = json.load(f)

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")


generation_config.json: 100%|██████████| 116/116 [00:00<00:00, 323kB/s]


In [5]:
#%%
abstracts = np.array(pmidUtils.process_abstracts_data(config, x))

# Grab Predictions & Test

## Zero-Shot Performance

In [10]:
# test_prompt = f"""
# 	Assessment Task:
# 	Conduct a through analysis of the provided biomedical abstract, and determine whether the text is relevant for evaluating the given hypothesis.
#     Consider the quality of the research, the relevance of the findings to the hypothesis, and the presence of any limitations or conflicting
#     evidence in your evaluation. 
    
#     Hypothesis:
#     Ezetimibe may effectively alleviate or target key pathogenic mechanisms of diabetes potentially offering therapeutic benefits or slowing disease progression.
    
#     Biomedical Abstract for Analysis:
#     {abstracts[0][0]}
    
# 	Format your response as:
# 	Score: [Number] - Reasoning: [Reasoning]
#     Scoring Guidelines:
# 		- 0: The abstract is not relevant and provides minimal useful information towards evaluating the given hypothesis.
# 		- 1: The abstract provides is relevant to the hypothesis and provides useful information towards evaluating the given hypothesis.
# """
test_prompt = f"""
	Is the hypothesis and provided abstract relevant for evaluating the accuracy of the hypothesis?
 
    Hypothesis:
    Ezetimibe may effectively alleviate or target key pathogenic mechanisms of diabetes potentially offering therapeutic benefits or slowing disease progression.
    
    Biomedical Abstract for Analysis:
    {abstracts[0][0]}
"""

In [11]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [13]:
inputs = tokenizer(test_prompt, return_tensors="pt")

In [16]:
output = model.generate(**inputs, do_sample = True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/w5home/lxu3/anaconda3/envs/kmGPT/lib/python3.11/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/w5home/lxu3/anaconda3/envs/kmGPT/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Input length of input_ids is 595, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [17]:
output

tensor([[    1, 28705,    13,    12,  2301,   272, 23592,   304,  3857, 11576,
          8598,   354,  6548,  1077,   272, 13252,   302,   272, 23592, 28804,
            13, 28705,    13,  2287,   382,  1416,  6317,   278, 28747,    13,
          2287,   413, 11456,   321, 17193,   993, 11466,   389,  2303, 13713,
           442,  2718,  1945,  2439,  8371,   294, 23400,   302, 22794, 13957,
          9055,  7454, 25334,   294,  7196,   442,  3944,   288,  8030,  5097,
           296, 28723,    13,   260,    13,  2287,  6345, 16812,   745, 14372,
           354, 19442, 28747,    13,  2287,  9632,  1138, 28747, 28705, 28750,
         28770, 28774, 28782, 28784, 28750, 28782, 28770,  7379, 28747,   415,
           907,  6503,   403,   298,  3190,  1944, 15627,   272, 11125,   298,
           690,  3923,   505, 11503,   338,  4721,   360,  4362,   366,   515,
           325, 28765, 28769, 28731,   349,   916,  5605,  3450,  2352,   304,
           916,  4110,   601, 28723,   415,  1676,  

In [18]:
tokenizer.batch_decode(output)

['<s> \n\tIs the hypothesis and provided abstract relevant for evaluating the accuracy of the hypothesis?\n \n    Hypothesis:\n    Ezetimibe may effectively alleviate or target key pathogenic mechanisms of diabetes potentially offering therapeutic benefits or slowing disease progression.\n    \n    Biomedical Abstract for Analysis:\n    PMID: 23956253 Text: The first aim was to critically evaluate the extent to which familial hypercholesterolaemia (FH) is underdiagnosed and undertreated. The second aim was to provide guidance for screening and treatment of FH, in order to prevent coronary heart disease (CHD). Of the theoretical estimated prevalence of 1/500 for heterozygous FH, <1% are diagnosed in most countries. Recently, direct screening in a Northern European general population diagnosed approximately 1/200 with heterozygous FH. All reported studies document failure to achieve recommended LDL cholesterol targets in a large proportion of individuals with FH, and up to 13-fold increa

## High Dimenstional Clusters?

## Fine-Tuned Model